# Introduction

### Inference Question:
How is the likelihood of contracting heart disease is impacted by 6 different health factors?

### The Heart Disease Dataset
The dataset that we will be using for this project is one related to heart disease and the factors that contribute towards it. Different factors such as blood pressure, cholesterol and even age can have great impacts on the risk of contracting heart disease and so this data set provides past details of patients with attributes they had and whether or not they were healthy or not. The link to this dataset is included below, but in this project we will be reading the data directly from the web, and so there is no need to have the file saved locally.

https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/

From the above link, the data from the Cleveland dataset will be used. The original dataset contains 76 factors, however, the processed data, which is what will be used in our project, contains 14 factors. We will further filter this data to be the 6 factors that we will be examining, which are listed below.

#### The Health Factors We Are Focusing On:
- age -> the age of the patient
- sex -> the sex of the patient
- pain.type -> the type of pain experienced by the patient (angina [1], abnormal angina [2], not angina [3], or asymptomatic [4])
- restbps -> the resting blood pressure of the patient
- cholesterol -> the cholesterol level of the patient
- maxbpm -> the maximum heart rate of the patient